#**Lab 10 -Transfer learning implementation using VGG16 model to classify images**

###**Importing Libraries**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

###**Data Preprocessing**

In [ ]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


###**Loading the VGG16 model**

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

###**Adding custom layers for our problem**

In [ ]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 8s 105ms/step - loss: 0.2175 - accuracy: 0.9246 - val_loss: 1.9222 - val_accuracy: 0.6965
Epoch 2/50
65/65 [==============================] - 7s 101ms/step - loss: 0.1777 - accuracy: 0.9387 - val_loss: 1.8048 - val_accuracy: 0.6965
Epoch 3/50
65/65 [==============================] - 7s 101ms/step - loss: 0.1345 - accuracy: 0.9538 - val_loss: 1.8878 - val_accuracy: 0.6946
Epoch 4/50
65/65 [==============================] - 7s 100ms/step - loss: 0.1645 - accuracy: 0.9455 - val_loss: 1.9112 - val_accuracy: 0.6770
Epoch 5/50
65/65 [==============================] - 7s 100ms/step - loss: 0.2066 - accuracy: 0.9338 - val_loss: 1.7842 - val_accuracy: 0.6887
Epoch 6/50
65/65 [==============================] - 7s 101ms/step - loss: 0.1658 - accuracy: 0.9455 - val_loss: 2.2073 - val_accuracy: 0.7218
Epoch 7/50
65/65 [==============================] - 7s 103ms/step - loss: 0.1093 - accuracy: 0.9582 - val_loss: 2.0057 - val_accuracy: 0.7121
Epoch 

###**Accuracy:97%**